In [0]:
%pip install --upgrade python-dotenv


Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import requests
import json
from pyspark.sql.functions import *
from pyspark.sql.window import *
from dotenv import load_dotenv

In [0]:
#Copying the api_params env file from file store/tables to a local instance
from dotenv import load_dotenv
import os
dbutils.fs.cp("dbfs:/FileStore/tables/api_params.env", "file:/tmp/your_file.env", True)
load_dotenv("/tmp/your_file.env")
dotenv_path = '/dbfs/FileStore/tables/api_params.env'
load_dotenv(dotenv_path)

Out[2]: False

In [0]:
API_KEY = os.getenv("API_KEY")
CHANNEL_NAME = 'rajshamani'
URL1 = f'https://www.googleapis.com/youtube/v3/search?part=snippet&type=channel&q={CHANNEL_NAME}&key={API_KEY}'

#Getting the channel id from the youtube's channel details API
CHANNEL_ID= ''
try:
    data1 = requests.get(URL1)
    data_json1 = data1.json()
    print(data_json1)
    CHANNEL_ID = data_json1['items'][0]['snippet']['channelId']

except Exception as e:
    print('Failed to Fetch', {e.error})


In [0]:
#Getting the upload playlist id of the channel, from youtube's content details api
UPLOAD_PLAYLIST_ID=''
URL2 = f'https://www.googleapis.com/youtube/v3/channels?part=contentDetails&id={CHANNEL_ID}&key={API_KEY}'
try:
    data2 = requests.get(URL2)
    data_json2 = data2.json()
    UPLOAD_PLAYLIST_ID = data_json2['items'][0]['contentDetails']['relatedPlaylists']['uploads']
except Exception as e:
    print('Failed to Fetch', {e})

In [0]:


#Get the latest 50 - video ids, of the videos in the uploads playlist

video_ids = []
next_page_token = ''
while True:
    URL3 = f'https://www.googleapis.com/youtube/v3/playlistItems?part=snippet&playlistId={UPLOAD_PLAYLIST_ID}&maxResults=50&pageToken={next_page_token}&key={API_KEY}'
    try:
        response = requests.get(URL3)
        data_json3 = response.json()

        for item in data_json3.get('items', []):
            video_ids.append(item['snippet']['resourceId']['videoId'])

        next_page_token = data_json3.get('nextPageToken')
        if not next_page_token:
            break  # No more pages

    except Exception as e:
        print('Failed to fetch videos:', e)
        break  # Exit loop on failure


In [0]:
video_details_data = []

for video_id in video_ids:
    try:
        VIDEO_DETAILS_URL = f'https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics&id={video_id}&key={API_KEY}'
        VIDEO_DETAILS = requests.get(VIDEO_DETAILS_URL)
        VIDEO_DETAILS_JSON = VIDEO_DETAILS.json()
        item = VIDEO_DETAILS_JSON.get('items', [{}])[0]

        snippet = item.get('snippet', {})
        statistics = item.get('statistics', {})

        channel_title = snippet.get('channelTitle')
        published_at = snippet.get('publishedAt')
        title = snippet.get('title')
        description = snippet.get('description')
        tags = str(snippet.get('tags')) if 'tags' in snippet else None
        view_count = int(statistics['viewCount']) if 'viewCount' in statistics else None
        like_count = int(statistics['likeCount']) if 'likeCount' in statistics else None
        comment_count = int(statistics['commentCount']) if 'commentCount' in statistics else None

        video_details_data.append((channel_title, published_at, title, description, tags, view_count, like_count, comment_count))

    except Exception as e:
        print('Failed to fetch', e)

In [0]:
columns = ['channel_Name', 'published_date', 'video_title', 'description', 'video_tags', 'view_count', 'like_count', 'comment_count']
df_ = spark.createDataFrame(video_details_data, columns)
df = df_.withColumn('published_date', to_date('published_date'))

df.display()